## Import required modules for loan calculation

This includes modules from pandas, numpy, datetime, and several bokeh components.

In [1]:
# import required modules
import pandas as pd
import numpy as np
from datetime import datetime as dt

# import bokeh modules
from bokeh.models import (ColumnDataSource, NumeralTickFormatter)
from bokeh.plotting import (figure, show, output_file)
from bokeh.io import output_notebook

The python generator below is used to generate the pandas dataframe for the amortization table. This code was taken from 
comment to the Practical Business blog post on Amortization tables. The link to the full code of the notebook
containing the python generator as well as the iterative code subsequently below can be found here on [Github](https://gist.github.com/sjmallon/e1ca2aee4574d5517b8d31c93832222a) by author 'sjmallon'. Link to the original post is
here [Practical Business Python](http://pbpython.com/amortization-model.html), by contributor Chris Muffitt.
The R language is where my strength lie but I'm trying to learn Python and for me the fastest way is to see how others do it then try to understand the mechanics behind it. The only changes made here were producing the plots via the interface to the Bokeh library as well allowing for user input for the start date.

In [2]:
def compute(principal, rate, payment, additional_payment):
    """
    Generator that yields successive period payment details
    """
    amount = payment + additional_payment
    while principal > 0:
        period_interest = round(principal * rate, 2)
        principal_reduction = min(principal, amount - period_interest)
        principal = principal - principal_reduction
        yield principal_reduction, period_interest, principal

## Create iterative code to generate mortgage/loan schedule
The iterative code will create separate dataframes for the three scenarios
that will have seven columns not including the index column.


In [3]:
def amortization_table(interest_rate, years, payments_year, principal, addl_principal=0, start_date=dt.today()):
    """
    Calculate the amortization schedule given the loan details


    :param interest_rate: The annual interest rate for this loan
    :param years: Number of years for the loan
    :param payments_year: Number of payments in a year
    :parma principal: Amount borrowed
    :param addl_principal (optional): Additional payments to be made each period. Default 0.
    :param start_date (optional): Start date. Default first of next month if none provided

    :return: 
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    #Calculate the fixed payment 
    period_rate = interest_rate/payments_year 
    payment = round(-1 * np.pmt(period_rate, years*payments_year, principal),2)
    
    #Generate the period details schedule
    df = pd.DataFrame(list(compute(principal, period_rate, payment, addl_principal)), columns=["Principal", "Interest", "Curr_Balance"])
    df["Additional Payment"] = addl_principal
    df["Total Payment"] = df["Principal"] + df["Interest"]
    df["Cummulative Principal"] = df["Principal"].cumsum()
    payment_dates = pd.date_range(start_date, periods=len(df.index), freq='MS')
    df.insert(0,'Payment_Date', payment_dates)
    df.index += 1
    df.index.name = "Period"
    
    #Create a summary statistics table
    payoff_date = df['Payment_Date'].iloc[-1]
    stats = pd.Series([payoff_date, interest_rate, years, principal, payment, addl_principal, df["Interest"].sum()],
                     index=["Payoff Date", "Interest Rate", "Years", "Principal", "Payment", "Additional Payment", "Total Interest"], )
    return df, stats

## Setup three scenarios
Three scenarios were created and the only variable tested was additonal payments made.
All other variables were kept constant.

### Table statistics for the three scenarios is shown as output

In [4]:
year = int(input('Enter 4-digit start year:')) # request user input for 4-digit year
month = int(input('Enter 2-digit start month:'))
day = int(input('Enter 2-digit start day:'))

schedule1, stats1 = amortization_table(0.0325, 30, 12, 94000, addl_principal=0, start_date = dt(year, month, day))
schedule2, stats2 = amortization_table(0.0325, 30, 12, 94000, addl_principal=75, start_date = dt(year, month, day))
schedule3, stats3 = amortization_table(0.0325, 30, 12, 94000, addl_principal=200, start_date = dt(year, month, day))

# define function to highlight the minimum total interest paid
# for all three scenarios in the pd.DataFrame
def highlight_min(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)

# Combine all the scenarios into 1 view
table = pd.DataFrame([stats1, stats2, stats3])
table.style.apply(highlight_min, subset=['Total Interest']).format({'Interest Rate':'{:.2%}',
                                                                    'Principal': '${:,.0f}',
                                                                    'Payment': '${:,.2f}',
                                                                   'Total Interest': '${:,.2f}'})

Enter 4-digit start year:2013
Enter 2-digit start month:07
Enter 2-digit start day:01


,Payoff Date,Interest Rate,Years,Principal,Payment,Additional Payment,Total Interest
0,2043-07-01 00:00:00,3.25%,30,"$94,000",$409.09,0,"$53,274.84"
1,2036-06-01 00:00:00,3.25%,30,"$94,000",$409.09,75,"$39,581.85"
2,2030-03-01 00:00:00,3.25%,30,"$94,000",$409.09,200,"$27,886.67"


## Combine all the scenarios into one plot

The two plots below displays all three scenarios. The first shows the trend of the loan balance and the second the trend of the 
total annual interest.

In [5]:
output_notebook()

# define plot plot attributes and legends
colors_list = ['#1f77b4','#ff7f0e','#2ca02c']
legends_list = ['scenario1', 'scenario2','scenario3']
xs=[schedule1['Payment_Date'], schedule2['Payment_Date'], schedule3['Payment_Date']]
ys=[schedule1['Curr_Balance'], schedule2['Curr_Balance'], schedule3['Curr_Balance']]


# set up the figure
p = figure(plot_width=600, plot_height=250, title='Payoff Timelines', x_axis_type='datetime')

p.xgrid.grid_line_color = None
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Loan Balance'
p.yaxis[0].formatter = NumeralTickFormatter(format='$0,0')

# make a for loop for the list of plot attributes above and then show()
for (colr, leg, x, y ) in zip(colors_list, legends_list, xs, ys):
    my_plot = p.line(x, y, color = colr, legend = leg, line_width = 2)

show(p)

Loading BokehJS ...

In [6]:
def GroupbyYear(y):
    y['Year'] = y['Payment_Date'].dt.year
    return y.groupby('Year', as_index=False).agg({'Interest': 'sum'})
        
t1 = GroupbyYear(schedule1)
t2 = GroupbyYear(schedule2)
t3 = GroupbyYear(schedule3)

output_notebook()

# define plot plot attributes and legends
colors_list = ['#1f77b4','#ff7f0e','#2ca02c']
legends_list = ['scenario1', 'scenario2','scenario3']
xs=[t1['Year'], t2['Year'], t3['Year']]
ys=[t1['Interest'], t2['Interest'], t3['Interest']]


# set up the figure
TOOLS="hover,crosshair"

p = figure(plot_width=600, plot_height=250, title='Total Annual Interest',
          tools=TOOLS, toolbar_location=None)

p.xgrid.grid_line_color = None
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Total Annual Amount'
p.yaxis[0].formatter = NumeralTickFormatter(format='$0,0')
p.legend.click_policy='hide'

# make a for loop for the list of plot attributes above and then show()
for (colr, leg, x, y ) in zip(colors_list, legends_list, xs, ys):
    my_plot = p.vbar(x=x, top=y, color = colr, legend = leg, width = 0.9)

show(p)

Loading BokehJS ...